In [7]:
import torch
from transformers import pipeline

In [8]:

classifier = pipeline(
    "text-classification",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

result = classifier("It's definitely a movie")
print(result)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use mps:0


[{'label': 'POSITIVE', 'score': 0.9998583793640137}]


In [9]:
from ml_ops_project.models import SentimentClassifier
from transformers import AutoTokenizer

# Load the model from the best checkpoint
# Note: Adjust the filename if you want a specific version
checkpoint_path = "../models/epoch-epoch=01-val_accuracy=0.842.ckpt"
model = SentimentClassifier.load_from_checkpoint(checkpoint_path)
model.eval()

# Load tokenizer (using the model name stored in hyperparameters if available, else default)
model_name = model.hparams["model_name"] if hasattr(model, "hparams") else "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Predict
text = "It's definitely a good movie"
inputs = tokenizer(text, return_tensors="pt")

# Move inputs to the same device as the model
inputs = {key: val.to(model.device) for key, val in inputs.items()}

with torch.no_grad():
    logits = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"]).logits
    predicted_class_id = logits.argmax().item()

print(f"Prediction: {predicted_class_id} (0=Negative, 1=Positive)", "Score: ", torch.softmax(logits, dim=1)[0, predicted_class_id].item())
print(f"Logits: {logits}")

Prediction: 1 (0=Negative, 1=Positive) Score:  0.980222761631012
Logits: tensor([[-1.9622,  1.9410]], device='mps:0')
